In [144]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [145]:
data = pd.read_csv('Fifa21_validate.csv')

In [146]:
def preprocess (data):
    data.columns = data.columns.str.lower()
    data['position'] = data['position'].fillna('CB')
    data = data.dropna()
    data = data.drop(['loan date end','bp','id','name','nationality','contract','club','team & contract','release clause','growth','joined','position','foot'], axis=1)
    data['height'] = data['height'].str.replace("'",".").str.replace("''","")
    data['height'] = data['height'].str.replace('"',"")
    data['weight'] = data['weight'].str.rstrip("lbs").astype(float)
    data['height'] = data['height'].astype(float)
    data['value'] = data['value'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')
    data['value'] = pd.to_numeric(data['value'])
    data['wage'] = data['wage'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')
    data['wage'] = pd.to_numeric(data['wage'])
    data['ls'] = data['ls'].str.split('+').str[0]
    
    return data 
    
modified_data = preprocess(data)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17236\53303489.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['value'] = data['value'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17236\53303489.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['wage'] = data['wage'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')


In [147]:
X_cat

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,2,2,Low,High,1,34,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16
33,3,3,Medium,Medium,1,6,57,57,57,63,62,62,62,63,63,63,63,63,58,58,58,63,51,48,48,48,51,47,40,40,40,47,16
37,3,3,High,Low,1,4,65,65,65,66,65,65,65,66,64,64,64,65,61,61,61,65,59,55,55,55,59,58,53,53,53,58,16
49,3,3,Medium,Medium,1,15,62,62,62,64,63,63,63,64,63,63,63,64,59,59,59,64,55,52,52,52,55,52,46,46,46,52,15
85,4,3,Medium,Medium,1,4,60,60,60,65,64,64,64,65,66,66,66,68,69,69,69,68,70,70,70,70,70,69,67,67,67,69,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,3,2,Medium,Medium,1,5,58,58,58,62,60,60,60,62,59,59,59,61,54,54,54,61,50,45,45,45,50,48,40,40,40,48,12
1863,3,3,Medium,Low,1,3,65,65,65,68,67,67,67,68,68,68,68,68,64,64,64,68,56,53,53,53,56,53,46,46,46,53,15
1872,3,3,High,Medium,1,13,72,72,72,67,70,70,70,67,68,68,68,65,64,64,64,65,54,56,56,56,54,52,55,55,55,52,17
1890,2,2,Medium,Medium,1,11,56,56,56,60,60,60,60,60,62,62,62,61,62,62,62,61,58,59,59,59,58,58,55,55,55,58,15


In [148]:
y = modified_data['ova'] #target variable
X = modified_data.drop(['ova'], axis=1) #dropped target variable
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [149]:
def preprocess2 (X_cat):
    for col, series in X_cat.iteritems():
        X_cat[col] = series.apply(lambda x: x.split('+')[0] if '+' in x else x)
    for col, series in X_cat.iteritems():
        X_cat[col] = series.apply(lambda x: x.replace('★', ' ') if '★' in x else x)
    X_cat = X_cat.apply(pd.to_numeric, errors = 'ignore')
    
    return X_cat

modified_data2 = preprocess2(X_cat)
dataframe = pd.concat([X_cat, X_num], axis=1)
    

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17236\3957567776.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in X_cat.iteritems():
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17236\3957567776.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in X_cat.iteritems():


In [150]:
dataframe

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,height,weight,value,wage,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy
0,2,2,Low,High,1,34,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,23,6.20,194.0,975000,5000,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74
33,3,3,Medium,Medium,1,6,57,57,57,63,62,62,62,63,63,63,63,63,58,58,58,63,51,48,48,48,51,47,40,40,40,47,16,21,5.70,148.0,575000,2000,261,58,57,38,63,45.0,290,65,56.0,47,56,66,369,77,74,73.0,61,84.0,301,58,69.0,66,47,61,269,65,32.0,50.0,62.0,60,58.0,77,26,24,27.0,52,14,8,12,6,12,1619,343,75,57,60,67,28,56
37,3,3,High,Low,1,4,65,65,65,66,65,65,65,66,64,64,64,65,61,61,61,65,59,55,55,55,59,58,53,53,53,58,16,21,5.70,161.0,11000000,800,309,66,69,60,64,50.0,302,67,62.0,45,61,67,366,75,71,77.0,60,83.0,306,66,65.0,61,54,60,281,45,52.0,65.0,55.0,64,53.0,141,43,45,53.0,54,13,12,10,8,11,1759,370,73,65,61,68,48,55
49,3,3,Medium,Medium,1,15,62,62,62,64,63,63,63,64,63,63,63,64,59,59,59,64,55,52,52,52,55,52,46,46,46,52,15,23,6.10,179.0,825000,700,303,66,61,43,65,68.0,332,68,75.0,68,56,65,338,70,74,72.0,60,62.0,322,73,53.0,70,69,57,251,52,35.0,55.0,57.0,52,59.0,111,34,36,41.0,49,9,11,9,12,8,1706,365,72,62,63,67,36,65
85,4,3,Medium,Medium,1,4,60,60,60,65,64,64,64,65,66,66,66,68,69,69,69,68,70,70,70,70,70,69,67,67,67,69,15,24,5.90,152.0,3000000,7000,262,65,30,64,72,31.0,337,72,63.0,58,72,72,353,73,71,72.0,67,70.0,350,65,68.0,89,71,57,326,72,65.0,63.0,63.0,63,65.0,196,67,65,64.0,42,5,12,9,7,9,1866,399,72,46,68,72,65,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,3,2,Medium,Medium,1,5,58,58,58,62,60,60,60,62,59,59,59,61,54,54,54,61,50,45,45,45,50,48,40,40,40,48,12,20,5.90,148.0,500000,1000,282,67,61,45,60,49.0,284,67,52.0,37,61,67,335,74,69,70.0,45,77.0,248,63,40.0,41,49,55,190,28,25.0,53.0,46.0,38,58.0,110,27,42,41.0,40,11,6,7,11,5,1489,329,71,58,57,67,34,42
1863,3,3,Medium,Low,1,3,65,65,65,68,67,67,67,68,68,68,68,68,64,64,64,68,56,53,53,53,56,53,46,46,46,53,15,27,5.10,163.0,1000000,8000,312,66,57,54,70,65.0,341,70,70.0,65,67,69,361,72,78,75.0,67,69.0,297,68,39.0,63,62,65,280,46,35.0,68.0,66.0,65,65.0,102,29,39,34.0,47,7,8,15,8,9,1740,369,75,62,68,70,36,58
1872,3,3,High,Medium,1,13,72,72,72,67,70,70,70,67,68,68,68,65,64,64,64,65,54,56,56,56,54,52,55,55,55,52,17,33,6.20,179.0,2000000,19000,341,51,75,76,66,73.0,292,65,57.0,49,46,75,299,51,58,65.0,72,53.0,381,78,80.0,74,82,67,313,71,35.0,76.0,68.0,63,70.0,118,59,35,24.0,53,11,9,14,13,6,1797,378,55,73,59,68,45,78
1890,2,2,Medium,Medium,1,11,56,56,56,60,60,60,60,60,62,62,62,61,62,62,62,61,58,59,59,59,58,58,55,55,55,58,15,21,6.10,163.0,525000,2000,275,59,49,49,68,50.0,309,60,59.0,57,68,65,309,61,52,66.0,60,70.0,271,66,47.0,60,51,47,284,49,47.0,55.0,69.0,64,59.0,173,57,58,58.0,45,12,13,6,7,7,1666,345,56,53,65,63,55,53


In [151]:
pd.set_option('display.max_columns', None)
dataframe

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,height,weight,value,wage,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy
0,2,2,Low,High,1,34,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,23,6.20,194.0,975000,5000,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,64,30,50,50,66,74
33,3,3,Medium,Medium,1,6,57,57,57,63,62,62,62,63,63,63,63,63,58,58,58,63,51,48,48,48,51,47,40,40,40,47,16,21,5.70,148.0,575000,2000,261,58,57,38,63,45.0,290,65,56.0,47,56,66,369,77,74,73.0,61,84.0,301,58,69.0,66,47,61,269,65,32.0,50.0,62.0,60,58.0,77,26,24,27.0,52,14,8,12,6,12,1619,343,75,57,60,67,28,56
37,3,3,High,Low,1,4,65,65,65,66,65,65,65,66,64,64,64,65,61,61,61,65,59,55,55,55,59,58,53,53,53,58,16,21,5.70,161.0,11000000,800,309,66,69,60,64,50.0,302,67,62.0,45,61,67,366,75,71,77.0,60,83.0,306,66,65.0,61,54,60,281,45,52.0,65.0,55.0,64,53.0,141,43,45,53.0,54,13,12,10,8,11,1759,370,73,65,61,68,48,55
49,3,3,Medium,Medium,1,15,62,62,62,64,63,63,63,64,63,63,63,64,59,59,59,64,55,52,52,52,55,52,46,46,46,52,15,23,6.10,179.0,825000,700,303,66,61,43,65,68.0,332,68,75.0,68,56,65,338,70,74,72.0,60,62.0,322,73,53.0,70,69,57,251,52,35.0,55.0,57.0,52,59.0,111,34,36,41.0,49,9,11,9,12,8,1706,365,72,62,63,67,36,65
85,4,3,Medium,Medium,1,4,60,60,60,65,64,64,64,65,66,66,66,68,69,69,69,68,70,70,70,70,70,69,67,67,67,69,15,24,5.90,152.0,3000000,7000,262,65,30,64,72,31.0,337,72,63.0,58,72,72,353,73,71,72.0,67,70.0,350,65,68.0,89,71,57,326,72,65.0,63.0,63.0,63,65.0,196,67,65,64.0,42,5,12,9,7,9,1866,399,72,46,68,72,65,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,3,2,Medium,Medium,1,5,58,58,58,62,60,60,60,62,59,59,59,61,54,54,54,61,50,45,45,45,50,48,40,40,40,48,12,20,5.90,148.0,500000,1000,282,67,61,45,60,49.0,284,67,52.0,37,61,67,335,74,69,70.0,45,77.0,248,63,40.0,41,49,55,190,28,25.0,53.0,46.0,38,58.0,110,27,42,41.0,40,11,6,7,11,5,1489,329,71,58,57,67,34,42
1863,3,3,Medium,Low,1,3,65,65,65,68,67,67,67,68,68,68,68,68,64,64,64,68,56,53,53,53,56,53,46,46,46,53,15,27,5.10,163.0,1000000,8000,312,66,57,54,70,65.0,341,70,70.0,65,67,69,361,72,78,75.0,67,69.0,297,68,39.0,63,62,65,280,46,35.0,68.0,66.0,65,65.0,102,29,39,34.0,47,7,8,15,8,9,1740,369,75,62,68,70,36,58
1872,3,3,High,Medium,1,13,72,72,72,67,70,70,70,67,68,68,68,65,64,64,64,65,54,56,56,56,54,52,55,55,55,52,17,33,6.20,179.0,2000000,19000,341,51,75,76,66,73.0,292,65,57.0,49,46,75,299,51,58,65.0,72,53.0,381,78,80.0,74,82,67,313,71,35.0,76.0,68.0,63,70.0,118,59,35,24.0,53,11,9,14,13,6,1797,378,55,73,59,68,45,78
1890,2,2,Medium,Medium,1,11,56,56,56,60,60,60,60,60,62,62,62,61,62,62,62,61,58,59,59,59,58,58,55,55,55,58,15,21,6.10,163.0,525000,2000,275,59,49,49,68,50.0,309,60,59.0,57,68,65,309,61,52,66.0,60,70.0,271,66,47.0,60,51,47,284,49,47.0,55.0,69.0,64,59.0,173,57,58,58.0,45,12,13,6,7,7,1666,345,56,53,65,63,55,53


In [152]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
dataframe['a/w']= labelencoder.fit_transform(dataframe['a/w'])
dataframe['d/w']=labelencoder.fit_transform(dataframe['d/w'])

In [153]:
# transformer = MinMaxScaler().fit(dataframe)
# x_normalized = transformer.transform(dataframe)
# print(x_normalized.shape)
# data_frame = x_normalized
# data_frame = pd.DataFrame(x_normalized, columns=dataframe.columns)
# data_frame
from sklearn.preprocessing import StandardScaler
Standardtransformer = StandardScaler().fit(dataframe)
X_standardized = Standardtransformer.transform(dataframe)
print(X_standardized.shape)
X_standardized = pd.DataFrame(X_standardized,columns=dataframe.columns)

(110, 87)


In [154]:
X = data_frame 

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [156]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(88, 87)
(22, 87)
(88,)
(22,)


In [157]:
X_train.head()

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,height,weight,value,wage,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy
65,0.50,0.333333,0.5,1.0,0.0,0.0625,0.964286,0.964286,0.964286,0.779661,0.847458,0.847458,0.847458,0.779661,0.810345,0.810345,0.810345,0.728814,0.754386,0.754386,0.754386,0.728814,0.526316,0.648148,0.648148,0.648148,0.526316,0.491228,0.625000,0.625000,0.625000,0.491228,0.161765,1.000000,0.6,0.602740,0.028977,0.325301,0.980328,0.550725,1.000000,0.973333,0.774194,1.000000,0.677419,0.782609,0.705882,0.328358,0.637931,0.840580,0.354701,0.162162,0.079365,0.360656,1.000000,0.636364,0.731707,0.821429,0.839286,0.191176,0.709091,0.680556,0.948276,0.910448,0.507692,1.000000,0.784314,0.958904,1.000000,0.331633,0.536232,0.289855,0.208955,0.125668,0.151899,0.188406,0.146667,0.054795,0.089744,0.734295,0.410112,0.000000,0.943396,0.595745,0.693878,0.410714,0.600
26,0.50,0.333333,0.0,1.0,0.0,0.0100,0.660714,0.660714,0.660714,0.576271,0.593220,0.593220,0.593220,0.576271,0.517241,0.517241,0.517241,0.559322,0.403509,0.403509,0.403509,0.559322,0.368421,0.296296,0.296296,0.296296,0.368421,0.350877,0.303571,0.303571,0.303571,0.350877,0.088235,0.111111,0.8,0.452055,0.001233,0.036145,0.524590,0.275362,0.657534,0.626667,0.322581,0.422535,0.374194,0.666667,0.264706,0.253731,0.034483,0.608696,0.521368,0.662162,0.777778,0.459016,0.000000,0.436364,0.629268,0.482143,0.517857,0.661765,0.600000,0.541667,0.513793,0.447761,0.153846,0.658228,0.392157,0.616438,0.379310,0.107143,0.144928,0.130435,0.119403,0.101604,0.101266,0.159420,0.080000,0.123288,0.051282,0.399830,0.196629,0.677966,0.547170,0.063830,0.489796,0.053571,0.550
22,0.75,0.666667,1.0,1.0,0.0,0.0100,0.803571,0.803571,0.803571,0.813559,0.779661,0.779661,0.779661,0.813559,0.775862,0.775862,0.775862,0.796610,0.701754,0.701754,0.701754,0.796610,0.614035,0.555556,0.555556,0.555556,0.614035,0.561404,0.446429,0.446429,0.446429,0.561404,0.088235,0.166667,0.6,0.547945,0.013564,0.132530,0.816393,0.753623,0.726027,0.600000,0.725806,0.816901,0.835484,0.869565,0.838235,0.791045,0.724138,0.811594,0.696581,0.756757,0.793651,0.622951,0.351351,0.618182,0.717073,0.589286,0.696429,0.529412,0.472727,0.791667,0.648276,0.432836,0.353846,0.721519,0.705882,0.698630,0.706897,0.290816,0.376812,0.217391,0.328358,0.088235,0.101266,0.057971,0.066667,0.136986,0.076923,0.706282,0.539326,0.745763,0.716981,0.680851,0.755102,0.250000,0.425
31,0.25,0.333333,1.0,1.0,0.0,0.0000,0.410714,0.410714,0.410714,0.372881,0.355932,0.355932,0.355932,0.372881,0.362069,0.362069,0.362069,0.406780,0.403509,0.403509,0.403509,0.406780,0.578947,0.611111,0.611111,0.611111,0.578947,0.614035,0.696429,0.696429,0.696429,0.614035,0.088235,0.055556,0.6,0.452055,0.001233,0.000000,0.409836,0.173913,0.232877,0.640000,0.516129,0.281690,0.345161,0.376812,0.250000,0.238806,0.413793,0.478261,0.474359,0.675676,0.634921,0.295082,0.054054,0.527273,0.482927,0.232143,0.750000,0.573529,0.672727,0.166667,0.448276,0.641791,0.646154,0.253165,0.137255,0.356164,0.379310,0.683673,0.637681,0.724638,0.686567,0.101604,0.126582,0.130435,0.066667,0.054795,0.128205,0.407470,0.185393,0.610169,0.075472,0.148936,0.204082,0.642857,0.650
47,0.50,0.333333,1.0,1.0,0.0,0.0175,0.535714,0.535714,0.535714,0.593220,0.542373,0.542373,0.542373,0.593220,0.534483,0.534483,0.534483,0.627119,0.526316,0.526316,0.526316,0.627119,0.701754,0.629630,0.629630,0.629630,0.701754,0.719298,0.642857,0.642857,0.642857,0.719298,0.073529,0.055556,0.6,0.452055,0.002774,0.036145,0.514754,0.608696,0.301

In [158]:
y_train.head()

1253    74
526     56
496     66
640     58
922     59
Name: ova, dtype: int64

In [159]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [160]:
predictions = lm.predict (X_train)
r2_score(y_train, predictions)

0.96973190747875

In [161]:
predictions

array([73.71920281, 56.5132314 , 66.03976322, 59.36740863, 58.26663592,
       67.70032906, 70.76283086, 57.86222956, 72.48563252, 66.68906532,
       53.69985008, 69.51996796, 71.23645996, 62.47896666, 67.09913057,
       64.62317677, 73.47238992, 56.45690117, 57.0827977 , 63.84671974,
       59.60167665, 65.22933921, 72.31153683, 54.21571385, 65.50889812,
       57.1453422 , 74.50319601, 65.27477621, 66.35709051, 62.21476603,
       64.68920357, 74.08492825, 66.11667435, 71.34692633, 67.93070465,
       73.6102166 , 60.61485945, 74.23818817, 62.10109325, 67.37024214,
       63.95824072, 58.72934093, 65.24229673, 70.3260739 , 63.43160196,
       63.08676164, 54.68474003, 77.2331598 , 66.97979048, 66.86193074,
       70.2439847 , 64.33594924, 72.64783247, 63.92952753, 68.28875945,
       70.26624063, 73.11080029, 62.66658988, 69.08299619, 64.24468811,
       66.08636041, 73.01569465, 65.1918758 , 60.55323722, 68.53232058,
       63.95851156, 71.68444523, 62.59683914, 58.93232365, 67.17

In [162]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

0.7017241381614605

In [163]:
y_test[:5]

1449    70
259     58
85      71
1528    65
1246    67
Name: ova, dtype: int64

In [164]:
predictions_test[:5]

array([70.16443082, 64.80202888, 69.35758964, 65.99427579, 68.83812314])

In [165]:
mae = mean_absolute_error(y_test, predictions_test)
print(mae)

2.2597714042209938


In [166]:
mse=mean_squared_error(y_test,predictions_test)
mse
rmse = math.sqrt(mse)  
print(rmse)  
mse 

2.9648794427516783


8.790510110051503

In [167]:
r2 = r2_score(y_test, predictions_test)
r2

0.7017241381614605

In [168]:
y_test.mean()

65.72727272727273

In [169]:
predictions.mean()

65.61363636363636